In [91]:
import sys
import logging

import numpy as np
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [138]:
import torch
import torch.nn.functional as F

from lrann.datasets import DataLoader, random_train_test_split
from lrann.estimators import ImplicitEst
from lrann.models import BilinearNet, DeepNet, ResNet, NaluNet, ResNetPlus, MoTBilinearNet
from lrann.evaluations import mrr_score, precision_recall_score
from lrann.utils import is_cuda_available

In [84]:
data = DataLoader().load_movielens('100k')

In [85]:
train, test = random_train_test_split(data)

## Estimator using a low-rank approximation model

In [120]:
lra_model = BilinearNet(data.n_users, data.n_items, embedding_dim=16, sparse=False)
lra_est = ImplicitEst(model=lra_model, 
                      n_iter=20, 
                      use_cuda=is_cuda_available())
lra_est.fit(train, verbose=True)

Epoch 0: loss 0.29848028162107604
Epoch 1: loss 0.152989525294814
Epoch 2: loss 0.12731886036375245
Epoch 3: loss 0.10986339597831438
Epoch 4: loss 0.09942853574164318
Epoch 5: loss 0.09429839696800085
Epoch 6: loss 0.08877062617923634
Epoch 7: loss 0.08440941384744058
Epoch 8: loss 0.08011859068543711
Epoch 9: loss 0.07848491075466403
Epoch 10: loss 0.07671918743031717
Epoch 11: loss 0.07268891464653181
Epoch 12: loss 0.07409993878517022
Epoch 13: loss 0.07247878540242342
Epoch 14: loss 0.0705476693850359
Epoch 15: loss 0.07078450364127117
Epoch 16: loss 0.0679047131916824
Epoch 17: loss 0.06707057180311148
Epoch 18: loss 0.066386374091455
Epoch 19: loss 0.06566404829926423


In [121]:
prec, recall = precision_recall_score(lra_est, test)
prec.mean(), recall.mean()

(0.08065573770491803, 0.0514430473199451)

In [122]:
mrr_score(lra_est, train).mean(), mrr_score(lra_est, test).mean()

(0.030054250525282745, 0.026500794537400105)

## Estimator using a deep neural model

In [51]:
nn_model = DeepNet(data.n_users, data.n_items, embedding_dim=8, sparse=False, activation=torch.tanh)
nn_est = ImplicitEst(model=nn_model, 
                     n_iter=20, 
                     use_cuda=is_cuda_available())
nn_est.fit(train, verbose=True)

Epoch 0: loss 0.23586470484969732
Epoch 1: loss 0.1894077275379335
Epoch 2: loss 0.18241314966638947
Epoch 3: loss 0.17853012318514797
Epoch 4: loss 0.1750466843886912
Epoch 5: loss 0.17116528122483646
Epoch 6: loss 0.17341163586795613
Epoch 7: loss 0.17019281759503904
Epoch 8: loss 0.1687646459923871
Epoch 9: loss 0.16890244743242128
Epoch 10: loss 0.16608317680479795
Epoch 11: loss 0.1675762428581809
Epoch 12: loss 0.1658183973630899
Epoch 13: loss 0.16381410580193373
Epoch 14: loss 0.16470830679447546
Epoch 15: loss 0.1637588477503101
Epoch 16: loss 0.163464301557391
Epoch 17: loss 0.16170240344742398
Epoch 18: loss 0.1607877084079144
Epoch 19: loss 0.1616262861316442


In [94]:
prec, recall = precision_recall_score(nn_est, test)
prec.mean(), recall.mean()

(0.07508196721311475, 0.04061013597008427)

In [95]:
mrr_score(nn_est, train).mean(), mrr_score(nn_est, test).mean()

(0.01895119212555599, 0.01881505176850961)

## Estimator using a deep residual model

In [98]:
res_model.h1_shift

Parameter containing:
tensor([-2.7856, -2.9772, -3.8177, -3.1163, -2.9332, -3.3910, -3.4420, -3.2846,
        -2.5489, -3.1643, -3.0984, -3.4706, -2.8285, -3.3326, -3.2923, -3.0912,
        -2.5761, -4.0115, -3.1415, -3.3916, -2.3225, -3.3407, -3.3187, -2.5934,
        -3.4852, -3.5608, -3.3923, -3.1715, -2.8499, -2.6917, -3.5449, -3.1505],
       requires_grad=True)

In [128]:
res_model = ResNetPlus(data.n_users, data.n_items, embedding_dim=16, sparse=False)
res_est = ImplicitEst(model=res_model, 
                     n_iter=20, 
                     use_cuda=is_cuda_available())


In [132]:
res_est.fit(train, verbose=True)

Epoch 0: loss 0.07194338748152199
Epoch 1: loss 0.07033568748498775
Epoch 2: loss 0.07072577106865291
Epoch 3: loss 0.06983841150237481
Epoch 4: loss 0.06877404474985392
Epoch 5: loss 0.06930768591382802
Epoch 6: loss 0.06584632108703835
Epoch 7: loss 0.06878036650550064
Epoch 8: loss 0.06637707301865667
Epoch 9: loss 0.06644411792789723
Epoch 10: loss 0.06482216810125699
Epoch 11: loss 0.06525102499783889
Epoch 12: loss 0.06360364171916917
Epoch 13: loss 0.06384760997495754
Epoch 14: loss 0.0648397792960191
Epoch 15: loss 0.06282563619876484
Epoch 16: loss 0.06354885548058006
Epoch 17: loss 0.06338949519225799
Epoch 18: loss 0.06251610719883688
Epoch 19: loss 0.06261655574637906


In [133]:
res_model.w

Parameter containing:
tensor([-1.4183], requires_grad=True)

In [134]:
prec, recall = precision_recall_score(res_est, test)
prec.mean(), recall.mean()

(0.0801639344262295, 0.04759151554330833)

In [135]:
mrr_score(res_est, train).mean(), mrr_score(res_est, test).mean()

(0.029556562088470826, 0.023649694391458316)

In [ ]:
nalu_model = MoTBilinearNet(data.n_users, data.n_items, embedding_dim=10, sparse=False)
nalu_est = ImplicitEst(model=nalu_model, 
                     n_iter=20, 
                     use_cuda=is_cuda_available())
nalu_est.fit(train, verbose=True)

Epoch 0: loss 0.3099866543679532
Epoch 1: loss 0.16082962157231315
Epoch 2: loss 0.13252661139859262
Epoch 3: loss 0.11672020119079131
Epoch 4: loss 0.10738558593765492
Epoch 5: loss 0.1023123516168345


In [150]:
prec, recall = precision_recall_score(nalu_est, test)
prec.mean(), recall.mean()

(0.07655737704918034, 0.04390486253096698)

In [151]:
torch.sigmoid(nalu_model.scaler_w)

tensor([0.9812, 0.9927, 0.9991, 0.9774, 0.9952, 0.9411, 0.9966, 0.9833, 0.9974,
        0.9947, 0.9683, 0.9986, 0.9382, 0.9951, 0.9706, 0.9577],
       grad_fn=<SigmoidBackward>)

In [152]:
mrr_score(nalu_est, train).mean(), mrr_score(nalu_est, test).mean()

(0.028476218185877214, 0.024242187465120288)